In [1]:
#| default_exp components

# Components

In [2]:
#| export
from fastcore.utils import *
from fastcore.xml import *
from fastcore.meta import use_kwargs, delegates

from dataclasses import dataclass, asdict

In [3]:
from pprint import pprint
from IPython import display

In [4]:
#| export
named = set('a button form frame iframe img input map meta object param select textarea'.split())
html_attrs = 'id cls title style'.split()
hx_attrs = 'get post put delete patch trigger target swap include select indicator push_url confirm disable replace_url on'
hx_attrs = html_attrs + [f'hx_{o}' for o in hx_attrs.split()]

In [5]:
#| export
@use_kwargs(html_attrs, keep=True)
def xt_html(tag: str, *c, **kwargs):
    tag,c,kw = xt(tag, *c, **kwargs)
    if tag in named and 'id' in kw and 'name' not in kw: kw['name'] = kw['id']
    return XT([tag,c,kw])

In [6]:
#| export
@use_kwargs(hx_attrs, keep=True)
def xt_hx(tag: str, *c, target_id=None, **kwargs):
    if target_id: kwargs['hx_target'] = '#'+target_id
    return xt(tag, *c, **kwargs)

In [7]:
#| export
_g = globals()
_all_ = ['Html', 'Head', 'Title', 'Meta', 'Link', 'Style', 'Body', 'Pre', 'Code',
    'Div', 'Span', 'P', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'Strong', 'Em', 'B',
    'I', 'U', 'S', 'Strike', 'Sub', 'Sup', 'Hr', 'Br', 'Img', 'Link', 'Nav',
    'Ul', 'Ol', 'Li', 'Dl', 'Dt', 'Dd', 'Table', 'Thead', 'Tbody', 'Tfoot', 'Tr',
    'Th', 'Td', 'Caption', 'Col', 'Colgroup', 'Form', 'Input', 'Textarea',
    'Button', 'Select', 'Option', 'Label', 'Fieldset', 'Legend', 'Details', 'Dialog',
    'Summary', 'Main', 'Header', 'Footer', 'Section', 'Article', 'Aside', 'Figure',
    'Figcaption', 'Mark', 'Small', 'Iframe', 'Object', 'Embed', 'Param', 'Video',
    'Audio', 'Source', 'Canvas', 'Svg', 'Math', 'Script', 'Noscript', 'Template', 'Slot']

for o in _all_: _g[o] = partial(xt_hx, o.lower())

In [8]:
def show_html(xt): return display.HTML(to_xml(xt))

In [9]:
#| export
picocss = "https://cdn.jsdelivr.net/npm/@picocss/pico@latest/css/pico.min.css"
picolink = Link(rel="stylesheet", href=picocss)

In [10]:
# Uncomment this and view in a notebook to see how these actually render with picocss
# show_html(picolink)

For tags that have a `name` attribute, it will be set to the value of `id` if not provided explicitly:

In [11]:
Form(Button(target_id='foo', id='btn'),
     hx_post='/', target_id='tgt', id='frm')

```html
<form hx-post="/" id="frm" hx-target="#tgt">
  <button id="btn" hx-target="#foo"></button>
</form>

```

In [12]:
#| export
@delegates(xt_hx, keep=True)
def A(*c, hx_get=None, target_id=None, hx_swap=None, href='#', **kwargs):
    return xt_hx('a', *c, href=href, hx_get=hx_get, target_id=target_id, hx_swap=hx_swap, **kwargs)

In [13]:
A('text', ht_get='/get', target_id='id')

```html
<a href="#" ht-get="/get" hx-target="#id">
text
</a>

```

In [14]:
#| export
@delegates(xt_hx, keep=True)
def AX(txt, hx_get=None, target_id=None, hx_swap=None, href='#', **kwargs):
    return xt_hx('a', txt, href=href, hx_get=hx_get, target_id=target_id, hx_swap=hx_swap, **kwargs)

In [15]:
AX('text', '/get', 'id')

```html
<a href="#" hx-get="/get" hx-target="#id">
text
</a>

```

In [19]:
#| export
@delegates(xt_hx, keep=True)
def Checkbox(checked:bool=False, label=None, **kwargs):
    if not checked: checked=None
    res = Input(type="checkbox", checked=checked, **kwargs)
    if label: res = Label(res, label)
    return res

In [20]:
show_html(Checkbox(True, 'Check me out!'))

In [21]:
#| export
@delegates(xt_hx, keep=True)
def Card(*c, header=None, footer=None, **kwargs):
    if header: c = (Header(header),) + c
    if footer: c += (Footer(footer),)
    return Article(*c, **kwargs)

In [22]:
show_html(Card('body', header=P('head'), footer=P('foot')))

In [23]:
#| export
@delegates(xt_hx, keep=True)
def Group(*c, **kwargs):
    return Fieldset(*c, role="group", **kwargs)

In [24]:
show_html(Group(Input(), Button("Save")))

In [25]:
#| export
@delegates(xt_hx, keep=True)
def Search(*c, **kwargs):
    return Form(*c, role="search", **kwargs)

In [26]:
show_html(Search(Input(type="search"), Button("Search")))

In [27]:
#| export
@delegates(xt_hx, keep=True)
def Grid(*c, cls='grid', **kwargs):
    c = tuple(o if isinstance(o,list) else Div(o) for o in c)
    return xt_hx('div', *c, cls=cls, **kwargs)

In [28]:
colors = [Input(type="color", value=o) for o in ('#e66465', '#53d2c5', '#f6b73c')]
show_html(Grid(*colors))

In [29]:
#| export
@delegates(xt_hx, keep=True)
def DialogX(*c, open=None, header=None, footer=None, id=None, **kwargs):
    card = Card(*c, header=header, footer=footer, **kwargs)
    return Dialog(card, open=open, id=id)

In [30]:
hdr = Div(Button(aria_label="Close", rel="prev"), P('confirm'))
ftr = Div(Button('Cancel', cls="secondary"), Button('Confirm'))
d = DialogX('thank you!', header=hdr, footer=ftr, open=None, id='dlgtest')
# use js or htmx to display modal

In [32]:
#| export
@delegates(xt_hx, keep=True)
def Hidden(value:str="", **kwargs):
    return Input(type="hidden", value=value, **kwargs)

In [33]:
#| export
def set_val(tag, attr, val):
    if attr.get('type', '') in ('checkbox','radio'):
        if val: attr['checked'] = '1'
        else: attr.pop('checked', '')
    else: attr['value'] = val

In [34]:
#| export
def find_inps(html):
    if not html: return []
    tag,cs,attrs = html
    if tag == 'input': return [html]
    res = []
    for c in cs:
        if isinstance(c, list): res.extend(find_inps(c))
    return res

In [35]:
#| export
def fill_form(form, obj):
    "Modifies form in-place and returns it"
    inps = find_inps(form)
    inps = {attrs['id']:(tag,attrs) for tag,c,attrs in inps if 'id' in attrs}
    for nm,val in asdict(obj).items():
        if nm in inps:
            tag,attr = inps[nm]
            set_val(tag, attr, val)
    return form

In [39]:
@dataclass
class TodoItem:
    title:str; id:int; done:bool
                
todo = TodoItem(id=2, title="Profit", done=True)
check = Checkbox(id="done", label='Done')
form = Form(Fieldset(Input(id="title"), check, Hidden(id="id"), Button("Save")))
fill_form(form, todo)

```html
<form>
  <fieldset>
    <input id="title" value="Profit">
    <label>
      <input type="checkbox" id="done" checked="1">
Done
    </label>
    <input type="hidden" value="2" id="id">
    <button>
Save
    </button>
  </fieldset>
</form>

```

In [40]:
#|export
def fill_dataclass(src, dest):
    "Modifies dataclass in-place and returns it"
    for nm,val in asdict(src).items(): setattr(dest, nm, val)
    return dest

In [41]:
nt = TodoItem('', 0, False)
fill_dataclass(todo, nt)
nt

TodoItem(title='Profit', id=2, done=True)

# Export -

In [42]:
#|hide
import nbdev; nbdev.nbdev_export()